In [1]:
from f_drosophila_infer import *
from f_train import *
from f_data_prep import *

In [2]:
data_all = np.loadtxt('../data_complete.txt')
data_all = data_all - np.median(data_all, axis=0)
all_bin = np.vsplit(data_all, 6)
all_init = np.vstack([all_bin[i] for i in range(5)])

complete_all = ([int(x) - 1 for x in open('../indices_complete.txt','r').readline().split()])
comp_ind = list(map(int, list((np.array(complete_all)[::6]-3)/6)))

In [3]:
data_comp = np.copy(data_all[:, comp_ind])
comp_bin = np.vsplit(data_comp, 6)
quad_comp = np.copy(data_comp)

for i in range(len(comp_ind)-1):
    for j in range(i+1, len(comp_ind)):
        quad_comp = np.hstack((quad_comp, (quad_comp[:,i]*quad_comp[:,j])[:,None]))
        
comp_quad_bin = np.vsplit(quad_comp, 6)
quad_init = np.vstack([comp_quad_bin[i] for i in range(5)])
comp_diff = np.vstack([comp_bin[i+1] - comp_bin[i] for i in range(5)])

print('# of quadratic predictors:', int(len(comp_ind)*(len(comp_ind)-1)/2))
print('Size of predictor:', quad_init.shape, '\nSize of target', comp_diff.shape)

# of quadratic predictors: 351
Size of predictor: (30390, 378) 
Size of target (30390, 27)


### alpha $*$ l1_ratio $*$ ||w||$_{1}$ + 0.5 $*$ alpha $*$ (1-l1_ratio) $*$ ||w||$^{2}_{2}$
- alpha = a + b
- l1_ratio = $\dfrac{a}{a+b}$

### fractional error = $\left[\dfrac{\displaystyle\sum_{cell, t} \left|g_{i, cell, t}^{pred} - g_{i, cell, t}^{data}\right|^{2}}{\displaystyle\sum_{cell, t} \left|g_{i, cell, t}^{data}\right|^{2}}\right]^{1/2}$

In [4]:
import timeit

start = timeit.default_timer()
w, sigma, bias = ER_quad_elastic(quad_init, comp_diff, power=1, alpha_=1, ratio_=0)
end = timeit.default_timer()

print(w.shape, bias.shape, sigma.shape)
print('simulation time: %s' % (end - start))

ridge_2727 = list([w, sigma, bias])
with open('./pickles/ridge_2727.pkl', 'wb') as f:
    pickle.dump(ridge_2727, f)
    

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


KeyboardInterrupt: 

In [ ]:
data_actual = np.vstack([comp_bin[i+1] for i in range(5)])
data_pred = all_init[:, comp_ind] + np.tanh(bias + quad_init.dot(w)) + sigma*npr.normal(size=(1,w.shape[1]))

E_comp = (np.sum(np.abs(data_pred - data_actual)**2, axis=0)/np.sum(np.abs(data_actual)**2, axis=0))**1/2

plt.figure(figsize=(12,6))
plt.plot(range(data_pred.shape[1]), E_comp, 'ro--')
# plt.plot(comp_ind, E[comp_ind], 'ko')
# plt.plot(comp_ind, comp_E, 'ro')
plt.xlabel('gene')
plt.ylabel('fractional error')
# plt.xlim(0,26)
# plt.ylim(0,1)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.linear_model import ElasticNet

grid = {'alpha': np.logspace(-1,2,4),\
            'l1_ratio' :[.2, .4, .6, .8]}
a=list(ParameterGrid(grid))

for i in range(len(a)):
    alpha = a[i]['alpha']
    ratio = a[i]['l1_ratio']
    w, sigma, bias = ER_quad_elastic(quad_init, comp_diff, power=1, alpha_=alpha, ratio_=ratio)
    
    data_actual = np.vstack([comp_bin[i+1] for i in range(5)])
    data_pred = all_init[:, comp_ind] + np.tanh(bias + quad_init.dot(w)) + sigma*npr.normal(size=(1,w.shape[1]))

    E_comp = (np.sum(np.abs(data_pred - data_actual)**2, axis=0)/np.sum(np.abs(data_actual)**2, axis=0))**1/2
    
    plt.figure(figsize=(12,6))
    plt.plot(range(data_pred.shape[1]), E_comp, 'ro--')
    # plt.plot(comp_ind, E[comp_ind], 'ko')
    # plt.plot(comp_ind, comp_E, 'ro')
    plt.xlabel('gene')
    plt.ylabel('fractional error')
    plt.title('alpha = %s, l1_ratio = %s, median = %s' % (alpha, ratio, round(np.percentile(E_comp, .75), 3)))
    # plt.xlim(0,26)
    # plt.ylim(0,1)
    plt.show()